In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from collections import namedtuple

import matplotlib.pyplot as plt 
import cv2 
import json
import numpy as np

import ritnet
import nia22
from glob import glob

from nia22 import scenario
from nia22.mask_utils import gen_mask
from nia22.eyes import Eye, device_dict, draw_two_eyes

In [3]:
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [4]:
ddir = '/home/hoseung/Work/NIA/data/jsons/'
vid_dir_base = '/home/hoseung/Work/NIA/data/run/by_id/'
#ddir = "/home/di/Work/data1/NIA2022/ORG/"
#vid_dir_base = "/mnt/syn2422/raw/raw/by_id/"

In [5]:
def get_all_scenario(uid, devices = ["V", "M"]):
    if not isinstance(uid, str): uid = f"{uid:03d}"
    pattern = ddir + f"{uid}/*.json"
    json_list = glob(pattern)
    json_list.sort()
    print(f"{len(json_list)} JSON files are found")
    
    ####
    all_scenarios = []
    for jl in json_list:
        jj = jl.split("/")[-1]
        device = jj.split("_")[5]
        code = jj[-14:-9]
        if device in devices:
        #if True:
            all_scenarios.append((device, code))

    scenario_set = set(all_scenarios)
    return scenario_set



RGB -> IR

기기에 따라 transformation 찾아내기.  
transformation은 뭘로 찾을까?  
ORB나 SWIFT feature 대신 mediapipe mesh끼리 비교. 


끝. 



In [6]:
uid = 71
scenario_set = get_all_scenario(uid)

for device, ss in scenario_set:
    print("Device", device_dict[device], "Scenario", ss)
    file = scenario.Info(device, base_dir = ddir + f"{uid:03d}/",
                         scen = ss)
    file.vid_dir = vid_dir_base + f"{file.id}/T1/{file.device_d}/RGB/"
    fn_vid = file.vid_dir + file.fn_vid
    
    break

0 JSON files are found


In [34]:
cap_r.release()
cap_i.release()

In [85]:
scenario_set

set()

In [38]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

In [77]:
if (cap_r := file.load_vid('rgb')) and (cap_i := file.load_vid('ir')):
    for ff, json in file:
        #eye = file.get_eye(json)
        #if not hasattr(eye, 'l_iris'):
        #    print(ff, "No iris detected")
        #    continue
        # read specific frame 
        cap_r.set(cv2.CAP_PROP_POS_FRAMES, ff)
        ok_rgb, rgb = cap_r.read()
        #rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        rgb = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
        cap_i.set(cv2.CAP_PROP_POS_FRAMES, ff)
        ok_ir, ir = cap_i.read()
        
        resi = face_mesh.process(ir)
        if resi.multi_face_landmarks:
            print("IR!")
            resr = face_mesh.process(rgb)
            if resr.multi_face_landmarks:
                break
        continue
        
        # IR to greyscale 혹은 R 밴드만? 
        #ir = cv2.cvtColor(ir, cv2.COLOR_BGR2GRAY)
        #ir = clahe.apply(ir)
        # then do something1111
        draw_two_eyes(rgb, eye, fn=f"{file.id}_{device}_{ff:03d}_rgb.png")
        draw_two_eyes(ir, eye, fn=f"{file.id}_{device}_{ff:03d}_ir.png")
        
else:
    print("Failed to open both videos")
    
    

9 No iris detected
IR!
IR!
32 No iris detected
33 No iris detected
34 No iris detected
IR!
54 No iris detected
75 No iris detected
79 No iris detected
83 No iris detected
87 No iris detected
119 No iris detected
121 No iris detected
IR!
140 No iris detected
145 No iris detected
160 No iris detected
172 No iris detected
197 No iris detected
IR!
211 No iris detected
IR!
IR!
245 No iris detected
IR!
257 No iris detected
IR!
275 No iris detected
283 No iris detected
291 No iris detected


In [75]:
resr.multi_face_landmarks

# 미디어파이프로 face mesh 찾기

face mesh 3D transform 모듈이 있음   
카메라 perspective 바꾸는 예제도 있을듯   
혹은 3D transform 자체에 대한 예제가 있을 수도.. 


In [62]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [45]:
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, 
                            refine_landmarks=True,
                            min_detection_confidence=0.5, 
                            min_tracking_confidence=0.5)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [65]:
save_annotated(rgb, resr, fn="rgb_mp_annotated.png")

TypeError: 'NoneType' object is not iterable

In [64]:
def save_annotated(image, results, fn):
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
        print('face_landmarks:', face_landmarks)
        mp_drawing.draw_landmarks(
              image=annotated_image,
              landmark_list=face_landmarks,
              connections=mp_face_mesh.FACEMESH_TESSELATION,
              landmark_drawing_spec=None,
              connection_drawing_spec=mp_drawing_styles
              .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
              image=annotated_image,
              landmark_list=face_landmarks,
              connections=mp_face_mesh.FACEMESH_CONTOURS,
              landmark_drawing_spec=None,
              connection_drawing_spec=mp_drawing_styles
              .get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
              image=annotated_image,
              landmark_list=face_landmarks,
              connections=mp_face_mesh.FACEMESH_IRISES,
              landmark_drawing_spec=None,
              connection_drawing_spec=mp_drawing_styles
              .get_default_face_mesh_iris_connections_style())
        cv2.imwrite(fn, annotated_image)

In [27]:
image = draw_two_eyes(rgb, eye, fn="test.png")

In [16]:
_2475192.jsonimage = draw_two_eyes(rgb, eye)
# Displaying the image

while(1):
    cv2.imshow('image', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break
         
cv2.destroyAllWindows()

AttributeError: 'Eye' object has no attribute 'l_iris'

## 2. JSON 로딩 

In [ ]:
anno = json.load(open(file._dir + fn_json,"r"))

## 3. RGB / IR 비교

In [25]:
eye = Eye(anno["Annotations"]["annotations"])

## 4. RGB -> IR 변환